# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

Adapted from https://github.com/nickmccullum/algorithmic-trading-python/blob/master/finished_files/002_quantitative_momentum_strategy.ipynb

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from statistics import mean

import datetime

import api_token  as gwg # handle iexcloud API tokens

In [2]:
env = "cloud" # "sandbox"  # 
BASE_URL, IEX_CLOUD_API_TOKEN = gwg.get_base_url(env=env), gwg.get_api_token(env=env)

BATCH_SIZE = 100

WRITE_ALL_DF = True
TOP_N = 20

file_csv = "stocks-ETF-sectors.csv"

filename=file_csv.replace(".csv", "")

# prepare output file-names
today = datetime.datetime.now()
xlsx_file = f"spreadsheet_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.xlsx"
xlsx_file_all = f"spreadsheet_{filename}_all_{today.strftime('%Y-%m-%d')}.xlsx"
watchlist_file = f"watchlist_{filename}_top{TOP_N}_{today.strftime('%Y-%m-%d')}.csv"

time_periods = [
    'Five-Day',
    'One-Month',
    'Three-Month',
    'Six-Month',
    'One-Year'
]
price_return_cols = [f'{time_period} Price Return' for time_period in time_periods]

# price_return_cols

hqm_columns = [
    'Ticker', 
    'Name',
    'Number of Shares to Buy', 
    'Price', 
    'Five-Day Price Return',
    'One-Month Price Return',
    'Three-Month Price Return',
    'Six-Month Price Return',
    'One-Year Price Return', 
    'Five-Day Return Percentile',
    'One-Month Return Percentile',
    'Three-Month Return Percentile',
    'Six-Month Return Percentile',
    'One-Year Return Percentile',
    'HQM Score'
]

## Load Stock Ticker

In [3]:
header = "Ticker, Name, Sector_File, TopN, WRITE_ALL"
df = pd.read_csv(
    file_csv, 
    names=[c.strip() for c in header.split(",")],
    skiprows=[0]
)

In [4]:
# df

In [5]:
stocks = df["Ticker"].values.tolist()

In [6]:
symbol_groups = list(gwg.chunks(stocks, BATCH_SIZE))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [7]:
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series([
                       symbol, 
                       data[symbol]['stats']['companyName'],
                       'N/A',
                       data[symbol]['quote']['latestPrice'],
                       data[symbol]['stats']['day5ChangePercent'],
                       data[symbol]['stats']['month1ChangePercent'],
                       data[symbol]['stats']['month3ChangePercent'],
                       data[symbol]['stats']['month6ChangePercent'],
                       data[symbol]['stats']['year1ChangePercent'],
                       'N/A',
                       'N/A',
                       'N/A',
                       'N/A',
                       'N/A',
                       'N/A'
                   ], 
                   index = hqm_columns
                ), 
                ignore_index = True
            )
        except Exception as ex:
            print(f"symbol = {symbol}\n Error={str(ex)}")

In [8]:
hqm_dataframe = hqm_dataframe.dropna(how='any', 
                subset=price_return_cols)

In [9]:
# hqm_dataframe[hqm_dataframe["One-Year Price Return"].isnull()]

## Calculating Momentum Percentiles

In [10]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = \
            0.01*stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])


## Calculating the HQM Score

The `HQM Score` will be the arithmetic mean of the momentum percentile scores 

In [11]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [12]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)

if WRITE_ALL_DF:
    hqm_dataframe_all = hqm_dataframe.copy()
    
hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

In [13]:
hqm_dataframe.reset_index(drop = True, inplace = True)

## Formatting .xlsx File

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [14]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
        'A': ['Ticker', string_template],
        'B': ['Name', string_template], 
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price', dollar_template],
        'E': ['Five-Day Price Return', percent_template], 
        'F': ['One-Month Price Return', percent_template], 
        'G': ['Three-Month Price Return', percent_template],
        'H': ['Six-Month Price Return', percent_template],
        'I': ['One-Year Price Return', percent_template],
        'J': ['Five-Day Return Percentile', percent_template],
        'K': ['One-Month Return Percentile', percent_template],
        'L': ['Three-Month Return Percentile', percent_template],
        'M': ['Six-Month Return Percentile', percent_template],
        'N': ['One-Year Return Percentile', percent_template], 
        'O': ['HQM Score', percent_template]
}

In [16]:
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [17]:
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [18]:
writer.save()

In [19]:
if WRITE_ALL_DF:
    writer = pd.ExcelWriter(xlsx_file_all, engine='xlsxwriter')
    hqm_dataframe_all.to_excel(writer, sheet_name='Momentum Strategy', index = False)

    for column in column_formats.keys():
        writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
        writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

    writer.save()

### save watchlist

In [20]:
tickers = hqm_dataframe[['Ticker']]
tickers.to_csv(watchlist_file, index=False)